In [ ]:
# !pip install transformers
! pip install -U git+https://github.com/huggingface/transformers.git
! pip install -U git+https://github.com/huggingface/accelerate.git

### Задание 14
Задание взять данные из https://www.kaggle.com/datasets/mrapplexz/bashim-quotes обучить модель GPT для генерации своих цитат

взять новостные данные из https://github.com/natasha/corus load_lenta2 нам понадобиться сам текст и заголовок обучить модель T5/ или GPT для генерации заголовков для статей

In [1]:
import pandas as pd

In [2]:
df_rec = pd.read_json('/kaggle/input/bashim-quotes/dataset.jsonl', lines=True).set_index('id')
df_rec.head()

,date,rating,text
id,,,
1,2004-08-30 11:24:00+00:00,22010.0,"<Ares> ppdv, все юниксы очень дружелюбны.. они..."
2,2004-08-30 11:25:00+00:00,25105.0,<томатик_рад> а ты не чувствуешь красоту мира?...
3,2004-08-30 11:27:00+00:00,7192.0,"<Дор> ""мышка, почему у тебя такие большие глаз..."
4,2004-08-30 11:28:00+00:00,29169.0,"<PPDV[os2]> ""Мальчики, вы что больные, бегать ..."
5,2004-08-30 11:26:00+00:00,7140.0,<Ohtori_Akio> мы - как разработчики - живём с ...


In [3]:
import re
from sklearn.model_selection import train_test_split

def build_text_files(data_json, dest_path):
    f = open(dest_path, 'w')
    data = ''
    for texts in data_json:
        summary = str(texts).strip()
        summary = re.sub(r"\s", " ", summary)
        data += summary + "  "
    f.write(data)

In [4]:
data = df_rec.loc[:5000, 'text']

In [5]:
train, test = train_test_split(data, test_size=0.15)

build_text_files(train,'train_dataset.txt')
build_text_files(test,'test_dataset.txt')

In [6]:
print("Train dataset length: "+ str(len(train)))
print("Test dataset length: "+ str(len(test)))

Train dataset length: 1197
Test dataset length: 212


In [7]:
from transformers import AutoTokenizer
#sberbank-ai/rugpt3large_based_on_gpt2
#sberbank-ai/rugpt3medium_based_on_gpt2
#sberbank-ai/rugpt3small_based_on_gpt2

tokenizer = AutoTokenizer.from_pretrained("sberbank-ai/rugpt3small_based_on_gpt2")

train_path = 'train_dataset.txt'
test_path = 'test_dataset.txt'

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [8]:
from transformers import TextDataset, DataCollatorForLanguageModeling

def load_dataset(train_path, test_path, tokenizer):
    train_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=train_path,
          block_size=128)

    test_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=test_path,
          block_size=128)

    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer, mlm=False,
    )
    return train_dataset, test_dataset, data_collator

train_dataset, test_dataset, data_collator = load_dataset(train_path, test_path, tokenizer)


/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

In [9]:
from transformers import Trainer, TrainingArguments, AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained("sberbank-ai/rugpt3small_based_on_gpt2")

In [10]:
training_args = TrainingArguments(
    output_dir="./gpt2-chief", #The output directory
    overwrite_output_dir=True, #overwrite the content of the output directory
    num_train_epochs=3, # number of training epochs
    per_device_train_batch_size=4, # batch size for training
    per_device_eval_batch_size=4,  # batch size for evaluation
    eval_steps = 400, # Number of update steps between two evaluations.
    save_steps=800, # after # steps model is saved
    warmup_steps=500,# number of warmup steps for learning rate scheduler
    )

In [11]:
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=test_dataset
)

In [12]:
trainer.train()

/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Step,Training Loss


TrainOutput(global_step=222, training_loss=4.141312057907517, metrics={'train_runtime': 286.5237, 'train_samples_per_second': 6.177, 'train_steps_per_second': 0.775, 'total_flos': 115621724160000.0, 'train_loss': 4.141312057907517, 'epoch': 3.0})

In [13]:
trainer.save_model()

In [14]:
tokenizer.save_pretrained('gpt_chf')

('gpt_chf/tokenizer_config.json',
 'gpt_chf/special_tokens_map.json',
 'gpt_chf/vocab.json',
 'gpt_chf/merges.txt',
 'gpt_chf/added_tokens.json',
 'gpt_chf/tokenizer.json')

In [15]:
tokenizer.save_pretrained('gpt_chf')
model.save_pretrained('model_gpt_chf')

In [16]:
tokenizer = AutoTokenizer.from_pretrained("gpt_chf")
model1 = AutoModelForCausalLM.from_pretrained("model_gpt_chf")

In [21]:
prefix = "Ехали медведи на велосипеде "

In [22]:
tokens = tokenizer(prefix, return_tensors='pt')
#tokens = {k: v.to(model.device) for k, v in tokens.items()}

In [23]:
size = tokens['input_ids'].shape[1]
output = model1.generate(
    **tokens, 
    #end_token=end_token_id,
    do_sample=False, 
    max_length=size+50, 
    repetition_penalty=5., 
    temperature=0.5,
    num_beams=10,
)

decoded = tokenizer.decode(output[0])
result = decoded[len(prefix):]
print(prefix + result)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Ехали медведи на велосипеде  <maxtar[away]> а кто-нить знает, где в Москве можно купить такую штуку? <Maxtar[away]> А то у меня есть один знакомый, который занимается продажей велосипедов. Он
